# read s3 delta table

the following code can be used to read data from data lake table stored on s3 in the delta file format (similar to parquet). 

we will read the dataframe using pyspark.

In [11]:
import os
import ast
import logging
import pprint
import yaml
import json
import time
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.column import *
import datetime as dt
import pandas as pd
import numpy as np

import boto3
import pprint
import botocore.session
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr

with open("creds.json", "r") as g:
    creds = json.load(g)
    g.close()

pp = pprint.PrettyPrinter(indent = 1)
# print("creds.json keys: ")
# pp.pprint([k for k in creds.keys()])

spark_host = creds["spark_host"]
spark_port = creds["spark_port"]
aws_client = creds["aws-access-key"]
aws_secret = creds["aws-secret-access-key"]
extra_jar_list = creds["extra_jar_list"]
print("imported modules.")

s3 = boto3.client("s3")
print("connected to boto3 clients.")

imported modules.
connected to boto3 clients.


## delta table s3 path

In [12]:
bucket = creds["bucket"]
subreddit = "aws"
filepath = os.path.join("s3a://", bucket, subreddit + "_clean")
print(filepath)

s3a://reddit-streaming-stevenhurwitt-new/aws_clean


## create spark session

In [7]:
try:
    spark = SparkSession.builder.appName("twitter") \
        .master("spark://{}:{}".format(spark_host, spark_port)) \
        .config("spark.executor.memory", "2048m") \
        .config("spark.executor.cores", "2") \
        .config("spark.streaming.concurrentJobs", "8") \
        .config("spark.local.dir", "/opt/workspace/tmp/driver/") \
        .config("spark.worker.dir", "/opt/workspace/tmp/executor/") \
        .config("spark.eventLog.enabled", "true") \
        .config("spark.eventLog.dir", "/opt/workspace/tmp/events/") \
        .config("spark.sql.debug.maxToStringFields", 1000) \
        .config("spark.jars.packages", extra_jar_list) \
        .config("spark.hadoop.fs.s3a.access.key", aws_client) \
        .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("spark.hadoop.fs.s3a.buffer.dir", "/opt/workspace/tmp/blocks") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
        .enableHiveSupport() \
        .getOrCreate()

    sc = spark.sparkContext
    # index = 0
    sc.setLogLevel("WARN")
    # sc.setLocalProperty("spark.scheduler.pool", "pool{}".format(str(index)))
    print("imported modules, created spark.")

except Exception as f:
    print("EXCEPTION: ")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/steven/.ivy2/cache
The jars for the packages stored in: /home/steven/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-34802d35-d071-49fd-b193-88bfeaf9fb7a;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in local-m2-cache
	found org.lz4#lz4-java;1.8.0 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in local-m2-cache
	found org.slf4j#slf4j-api;1.7.32 in local-m2-cache
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in local-m2-cache
	fo

EXCEPTION: 


## read df

In [8]:
try:
    df = spark.read.format("delta").option("header", True).load(filepath)
    # df.show()
    print("read df.")

except Exception as g:
    print("EXCEPTION: {}".format(g))

EXCEPTION: name 'spark' is not defined


## pandas df

In [ ]:
df_pandas = df.toPandas()
df_pandas

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,...,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,date,year,month,day
0,NaT,aws,I have this PHP (Laravel) script to get all th...,t2_6j0lp,False,None,0,False,Weird behavior getting a list of S3 object keys,r/aws,...,https://www.reddit.com/r/aws/comments/10t6cw6/...,228560,2023-02-04 05:48:13.401088,0,None,False,2023-02-04,2023,2,4
1,NaT,aws,I'm not exactly on the up and up on some of th...,t2_jsvj0raw,False,None,0,False,Pattern for ingesting deltas and merging into ...,r/aws,...,https://www.reddit.com/r/aws/comments/10pmcs3/...,228234,2023-01-31 03:00:16.421888,0,None,False,2023-01-31,2023,1,31
2,NaT,aws,"Hey there,\n\nIs anyone using the AWS Transfer...",t2_4b72e,False,None,0,False,AWS Transfer Family price question,r/aws,...,https://www.reddit.com/r/aws/comments/10p0t6t/...,228218,2023-01-30 12:32:19.943424,0,None,False,2023-01-30,2023,1,30
3,NaT,aws,"I can't imagine the answer is ""it does scale"",...",t2_52j9a,False,None,0,False,How does (does?) API Keys / Usage Plans / WAF ...,r/aws,...,https://www.reddit.com/r/aws/comments/10p3zl9/...,228191,2023-01-30 14:28:39.265280,0,None,False,2023-01-30,2023,1,30
4,NaT,aws,"Hi,\n\nI’m looking at using SES for email list...",t2_3sde0,False,None,0,False,SES Number of Contact Lists Per Account,r/aws,...,https://www.reddit.com/r/aws/comments/10opryt/...,228222,2023-01-30 02:08:13.197312,0,None,False,2023-01-30,2023,1,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,NaT,aws,I have data in a DDB. I want to make visualiza...,t2_uf837,False,None,0,False,Optimal way to visualize your data from DDB,r/aws,...,https://www.reddit.com/r/aws/comments/10s3qcg/...,228478,2023-02-02 22:58:15.340032,0,None,False,2023-02-02,2023,2,2
70,NaT,aws,"So, I'm trying to learn how to use ECS to port...",t2_6jr6y,False,None,0,False,Cloudformation: Is it just really bad for ever...,r/aws,...,https://www.reddit.com/r/aws/comments/10pjtk0/...,228228,2023-01-31 01:03:57.100032,0,None,False,2023-01-31,2023,1,31
71,NaT,aws,I created a [public hosted zone](https://i.img...,t2_e8so8,False,None,0,False,Why can't I see my domain's nameservers hosted...,r/aws,...,https://www.reddit.com/r/aws/comments/10t7vbt/...,228564,2023-02-04 07:19:56.327936,0,None,False,2023-02-04,2023,2,4
72,NaT,aws,I have a project to add ocr and search functio...,t2_ia3tn,False,None,0,False,search document text,r/aws,...,https://www.reddit.com/r/aws/comments/10t349w/...,228554,2023-02-04 02:53:44.418304,0,None,False,2023-02-04,2023,2,4


In [ ]:
df_pandas.shape

(74, 103)